<a href="https://colab.research.google.com/github/MariiaZimokha/notebooks/blob/main/kaggle/Cryptocurrency_Price_Prediction_with_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta

In [2]:
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=730)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('BTC-USD',
                      start=start_date,
                      end=end_date,
                      progress=False)

In [3]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-08-10,46280.847656,46637.988281,44705.554688,45585.031250,45585.031250,33546019517
2021-08-11,45599.703125,46735.632812,45351.710938,45593.636719,45593.636719,34319709073
2021-08-12,45576.878906,46228.910156,43861.445312,44428.289062,44428.289062,33723620826
2021-08-13,44439.691406,47831.976562,44282.417969,47793.320312,47793.320312,31744259539
2021-08-14,47810.687500,48098.683594,46177.632812,47096.945312,47096.945312,31211354442
...,...,...,...,...,...,...
2023-08-05,29075.388672,29102.464844,28957.796875,29042.126953,29042.126953,6598366353
2023-08-06,29043.701172,29160.822266,28963.833984,29041.855469,29041.855469,7269806994
2023-08-07,29038.513672,29244.281250,28724.140625,29180.578125,29180.578125,13618163710


In [4]:
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)
print(data.head())

        Date          Open          High           Low         Close  \
0 2021-08-10  46280.847656  46637.988281  44705.554688  45585.031250   
1 2021-08-11  45599.703125  46735.632812  45351.710938  45593.636719   
2 2021-08-12  45576.878906  46228.910156  43861.445312  44428.289062   
3 2021-08-13  44439.691406  47831.976562  44282.417969  47793.320312   
4 2021-08-14  47810.687500  48098.683594  46177.632812  47096.945312   

      Adj Close       Volume  
0  45585.031250  33546019517  
1  45593.636719  34319709073  
2  44428.289062  33723620826  
3  47793.320312  31744259539  
4  47096.945312  31211354442  


In [5]:
data.shape

(730, 7)

In [6]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,730.000000,730.000000,730.000000,730.000000,730.000000,7.300000e+02
mean,32436.901279,33083.053834,31726.725431,32408.800254,32408.800254,2.765164e+10
std,12878.321484,13193.197215,12505.003374,12859.196998,12859.196998,1.211319e+10
min,15782.300781,16253.047852,15599.046875,15787.284180,15787.284180,6.481776e+09
25%,21527.707031,21803.264648,20957.500488,21529.596191,21529.596191,1.866126e+10
50%,29031.107422,29386.275391,28455.166992,29024.082031,29024.082031,2.702928e+10
75%,42398.595703,43172.935547,41435.375000,42371.426758,42371.426758,3.376099e+10
max,67549.734375,68789.625000,66382.062500,67566.828125,67566.828125,1.189925e+11


In [7]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"],
                                        high=data["High"],
                                        low=data["Low"],
                                        close=data["Close"])])
figure.update_layout(title = "Bitcoin Price Analysis",
                     xaxis_rangeslider_visible=False)
figure.show()

In [8]:
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

Close        1.000000
Adj Close    1.000000
Low          0.998330
High         0.998301
Open         0.996197
Volume       0.199202
Name: Close, dtype: float64


<ipython-input-8-81091e368782>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [21]:
from sklearn.model_selection import train_test_split

X = data.drop(["Close", "Adj Close", "Date"], axis=1)
y = data["Close"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [9]:
import xgboost

In [25]:
model = xgboost.XGBRegressor(n_estimators=15, max_depth=20, verbosity=2)
model.fit(X_train, y_train)

[10:17:48] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 8 extra nodes, 0 pruned nodes, max_depth=3
[10:17:48] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 10 extra nodes, 0 pruned nodes, max_depth=3
[10:17:48] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 8 extra nodes, 0 pruned nodes, max_depth=3
[10:17:48] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 12 extra nodes, 0 pruned nodes, max_depth=3
[10:17:48] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 12 extra nodes, 0 pruned nodes, max_depth=4
[10:17:48] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=4
[10:17:48] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 12 extra nodes, 0 pruned nodes, max_depth=3
[10:17:48] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 20 extra nodes, 0 pruned nodes, max_depth=5
[10:17:48] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 22 extra nodes, 0 pruned nodes, max_de

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=20, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=15, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [20]:
model.score(X_test, y_test)
# model without Volume field

0.986951429154559

In [26]:
model.score(X_test, y_test)

0.9946075669197079

### Different model

**AutoTS** means **Automatic Time Series**, it is a machine learning library in Python used for the task of time series forecasting

Some of the popular features of the AutoTS library in Python are:

1. It can be used to find the best time series forecasting model which depends on the type of data you are using.
2. It can h ndle both the univariate and multivariate time series.
3. It can also handle messy data by removing and filling the NaN values and it can also handle the outliers.
4. You can use the models provided by this Python library for deployment also.


In [ ]:
!pip install autots

In [29]:
from autots import AutoTS
model = AutoTS(forecast_length=30, frequency='infer', ensemble='simple')
model = model.fit(data, date_col='Date', value_col='Close', id_col=None)
prediction = model.predict()
forecast = prediction.forecast
print(forecast)

Data frequency is: D, used frequency is: D
Model Number: 1 with model ARIMA in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model AverageValueNaive in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10
Model Number: 6 with model DatepartRegression in generation 0 of 10


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



Model Number: 7 with model DatepartRegression in generation 0 of 10


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



Model Number: 8 with model DatepartRegression in generation 0 of 10
Epoch 1/50
22/22 [==============================] - 10s 7ms/step - loss: 0.4171
Epoch 2/50
22/22 [==============================] - 0s 7ms/step - loss: 0.4150
Epoch 3/50
22/22 [==============================] - 0s 6ms/step - loss: 0.4157
Epoch 4/50
22/22 [==============================] - 0s 7ms/step - loss: 0.4151
Epoch 5/50
22/22 [==============================] - 0s 6ms/step - loss: 0.4160
Epoch 6/50
22/22 [==============================] - 0s 6ms/step - loss: 0.4147
Epoch 7/50
22/22 [==============================] - 0s 6ms/step - loss: 0.4145
Epoch 8/50
22/22 [==============================] - 0s 6ms/step - loss: 0.4141
Epoch 9/50
22/22 [==============================] - 0s 6ms/step - loss: 0.4137
Epoch 10/50
22/22 [==============================] - 0s 5ms/step - loss: 0.4160
Epoch 11/50
22/22 [==============================] - 0s 5ms/step - loss: 0.4143
Epoch 12/50
22/22 [==============================] - 0s 5ms/

...

Model Number: 168 of 191 with model UnivariateRegression for Validation 3
168 - UnivariateRegression with avg smape 3.77: 
Model Number: 169 of 191 with model UnivariateRegression for Validation 3
169 - UnivariateRegression with avg smape 3.77: 
Model Number: 170 of 191 with model SeasonalNaive for Validation 3
170 - SeasonalNaive with avg smape 3.14: 
Model Number: 171 of 191 with model WindowRegression for Validation 3
171 - WindowRegression with avg smape 5.42: 
Model Number: 172 of 191 with model WindowRegression for Validation 3
172 - WindowRegression with avg smape 3.99: 
Model Number: 173 of 191 with model WindowRegression for Validation 3


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



173 - WindowRegression with avg smape 12.24: 
Model Number: 174 of 191 with model WindowRegression for Validation 3
174 - WindowRegression with avg smape 5.38: 
Model Number: 175 of 191 with model GLM for Validation 3
175 - GLM with avg smape 3.58: 
Model Number: 176 of 191 with model WindowRegression for Validation 3
176 - WindowRegression with avg smape 4.74: 
Model Number: 177 of 191 with model WindowRegression for Validation 3
177 - WindowRegression with avg smape 4.12: 
Model Number: 178 of 191 with model WindowRegression for Validation 3
178 - WindowRegression with avg smape 4.19: 
Model Number: 179 of 191 with model SeasonalNaive for Validation 3
179 - SeasonalNaive with avg smape 4.05: 
Model Number: 180 of 191 with model SeasonalNaive for Validation 3
180 - SeasonalNaive with avg smape 6.92: 
Model Number: 181 of 191 with model SeasonalNaive for Validation 3


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.



181 - SeasonalNaive with avg smape 9.71: 
Model Number: 182 of 191 with model ETS for Validation 3
182 - ETS with avg smape 5.05: 
Model Number: 183 of 191 with model UnivariateRegression for Validation 3
183 - UnivariateRegression with avg smape 6.94: 
Model Number: 184 of 191 with model SeasonalNaive for Validation 3
184 - SeasonalNaive with avg smape 12.7: 
Model Number: 185 of 191 with model SeasonalNaive for Validation 3
185 - SeasonalNaive with avg smape 14.68: 
Model Number: 186 of 191 with model SeasonalNaive for Validation 3
186 - SeasonalNaive with avg smape 6.27: 
Model Number: 187 of 191 with model UnivariateRegression for Validation 3
187 - UnivariateRegression with avg smape 4.02: 
Model Number: 188 of 191 with model SeasonalNaive for Validation 3
188 - SeasonalNaive with avg smape 4.24: 
Model Number: 189 of 191 with model GLM for Validation 3
189 - GLM with avg smape 5.05: 
Model Number: 190 of 191 with model UnivariateRegression for Validation 3
190 - UnivariateRegress